### Health Check

In [1]:
[ordered]@{"$((Get-TimeZone).Id)" = Get-Date
    'Boot'      = (Get-CimInstance win32_operatingsystem).lastbootuptime.ToString('ddd M.d.yy H:m')
    'CpuUsage%' = (Get-CimInstance win32_processor | Measure-Object LoadPercentage -Average).Average
    'RamUsage%' = {$os=Get-CimInstance win32_operatingsystem; [Math]::Ceiling(($os.TotalVisibleMemorySize-$os.FreePhysicalMemory)*100/$os.TotalVisibleMemorySize)}.Invoke()
    'C:%Free'   = { $disk = Get-CimInstance Win32_LogicalDisk -Filter "DeviceID='C:'"; [Math]::Ceiling(($disk.FreeSpace / $disk.Size) * 100) }.Invoke()
            } | ft -HideTableHeaders -AutoSize


Central Standard Time 12/3/2023 2:53:11 PM
Boot                  Sun 12.3.23 11:9
CpuUsage%             79
RamUsage%             {48}
C:%Free               {55}



This shows which processes are consuming the most cpu, memory and network and have the most number of handles and threads currently on the system.

In [3]:
#$Host.UI.RawUI.BufferSize = New-Object Management.Automation.Host.Size(150, $Host.UI.RawUI.BufferSize.Height)  #v2: One line equivalent

$objArr = Get-Process | Select-Object @{Name='Process'; Expression={$_.ProcessName+'►'+$_.Id}}, CPU, ID, WS, HandleCount, Threads
# Sort for each property separately
$sortedByCPU = $objArr | Sort-Object CPU -Descending | Select-Object -First 10
$sortedByWS = $objArr | Sort-Object WS -Descending | Select-Object -First 10
$sortedByHandles = $objArr | Sort-Object HandleCount -Descending | Select-Object -First 10
$sortedByThreads = $objArr | Sort-Object Threads -Descending | Select-Object -First 10

$Network = (Get-Counter '\Process(*)\IO Read Operations/sec').CounterSamples.Where({$_.InstanceName -ne "_Total" -and $_.InstanceName -ne "Idle"})|
                sort cookedvalue -Descending|Select -Property instancename -First 10

$harray = for ($j = 0; $j -lt 10; $j++) {
    [PSCustomObject][ordered]@{
        'CPU'     = $sortedByCPU[$j].Process
        'WS'      = $sortedByWS[$j].Process
        'Handles' = $sortedByHandles[$j].Process
        'Threads' = $sortedByThreads[$j].Process
        'Network' = $Network[$j].InstanceName
    }
}
$harray | Format-Table -AutoSize


CPU                     WS                      Handles                 Threads                 Network
---                     --                      -------                 -------                 -------
msedge►13512            msedge►7464             OneDrive►11916          System►4                code
OneDrive►11916          dotnet►5428             System►4                Memory Compression►2632 code
msedge►6124             Code►5344               explorer►6532           explorer►6532           code
Code►5344               dwm►772                 msedge►7464             msedge►7464             code
msedge►7464             Grammarly.Desktop►12308 lsass►904               SearchHost►6964         svchost
Code►16052              Memory Compression►2632 Grammarly.Desktop►12308 Code►11620              dwm
Grammarly.Desktop►12308 SearchHost►6964         MonitoringHost►4256     msedgewebview2►14164    grammarly.desktop
Code►11620              svchost►1364            SearchHost►6964      